# **OpenWeather API**
To fetch live weather updates of any country or city by using the API code. 

# **Prerequisites**
Install the Python libraries on your local machine:
1. requests for making API calls
2. pandas for handling and manipulating data
3. pytz for handling timezones
4. csv library for writing data to CSV files

Skip these steps if you have installed all the libraries.

In [ ]:
# Open a new terminal and run the following command:
pip install requests 
pip install pandas
pip install pytz

# **Fetching OpenWeatherMap data from API**
The OpenWeatherMap API allows you to fetch OpenWeatherMap data in different formats:
1. Current weather
2. 3-hour Forecast 5 days

In [2]:
import requests
import pandas as pd

def fetch_weather_data(city):
    api_key = 'ceb3c45468b0de7eedac46058a1746f4'  # Replace with your actual OpenWeather API key
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={api_key}'

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def main():
    city = input("Enter city name: ")
    
    weather_data = fetch_weather_data(city)
    
    if weather_data:
        # Create a DataFrame to display weather data in a tabular format
        df = pd.DataFrame({
            'City': [weather_data['name']],
            'Temperature (°C)': [weather_data['main']['temp']],
            'Description': [weather_data['weather'][0]['description']],
            'Feels Like (°C)': [weather_data['main']['feels_like']],
            'Humidity (%)': [weather_data['main']['humidity']],
            'Pressure (hPa)': [weather_data['main']['pressure']],
            'Wind Speed (m/s)': [weather_data['wind']['speed']]
        })
        
        print("\nWeather Data Table:")
        print(df)
    else:
        print("Could not retrieve weather data.")

if __name__ == "__main__":
    main()


Weather Data Table:
       City  Temperature (°C) Description  Feels Like (°C)  Humidity (%)  \
0  New York             17.52   clear sky            16.55            47   

   Pressure (hPa)  Wind Speed (m/s)  
0            1010              6.69  


# **Fetching 5 day / 3 hour forecast data**
You can search weather forecast for 5 days with data every 3 hours by location.

In [7]:
import requests
import pandas as pd
from datetime import datetime

def fetch_weather_data(city, api_key):
    # Define the API URL for the 5 day / 3 hour forecast
    api_url = "https://api.openweathermap.org/data/2.5/forecast"
    
    # Set up the parameters for the API request
    params = {
        'q': city,                                      # City name
        'appid': 'ceb3c45468b0de7eedac46058a1746f4',    # Your OpenWeatherMap API key
        'units': 'metric'                               # Specify metric units for temperature
    }
    
    # Make the GET request
    response = requests.get(api_url, params=params)
    
    # Check if the response is successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

def extract_weather_info(data):
    """Extract relevant weather information and store in a DataFrame."""
    weather_records = []

    # Loop through the list of forecasts
    for entry in data['list']:
        timestamp = entry['dt_txt']
        temperature = entry['main']['temp']
        feels_like = entry['main']['feels_like']
        pressure = entry['main']['pressure']
        humidity = entry['main']['humidity']
        weather_description = entry['weather'][0]['description']
        wind_speed = entry['wind']['speed']
        rain_amount = entry.get('rain', {}).get('3h', 0)  # Get rainfall in last 3 hours, default to 0 if not present

        # Append data to the list
        weather_records.append({
            'Date & Time': timestamp,
            'Temperature (°C)': temperature,
            'Feels Like (°C)': feels_like,
            'Pressure (hPa)': pressure,
            'Humidity (%)': humidity,
            'Weather Description': weather_description,
            'Wind Speed (m/s)': wind_speed,
            'Rainfall (mm)': rain_amount
        })

    # Create a DataFrame from the records
    df = pd.DataFrame(weather_records)
    return df

# Main script
if __name__ == "__main__":
    city = input("Enter the city name: ")  # Prompt user to enter the city name
    api_key = "ceb3c45468b0de7eedac46058a1746f4"  # Replace with your OpenWeatherMap API key

    # Fetch the weather data
    weather_data = fetch_weather_data(city, api_key)

    # Check if data was fetched successfully and extract relevant information
    if weather_data:
        weather_df = extract_weather_info(weather_data)

        # Print the selected city along with the DataFrame
        print(f"Weather Forecast Data for {city}:")
        print(weather_df)

        # Optional: Save DataFrame to a CSV file
        weather_df.to_csv(f'Weather_Data_{city}_{pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv', index=False)


Weather Forecast Data for Singapore:
            Date & Time  Temperature (°C)  Feels Like (°C)  Pressure (hPa)  \
0   2024-11-18 18:00:00             25.13            26.15            1008   
1   2024-11-18 21:00:00             25.40            26.37            1008   
2   2024-11-19 00:00:00             25.80            26.73            1009   
3   2024-11-19 03:00:00             27.58            31.07            1010   
4   2024-11-19 06:00:00             27.13            30.34            1008   
5   2024-11-19 09:00:00             27.00            29.93            1006   
6   2024-11-19 12:00:00             26.43            26.43            1008   
7   2024-11-19 15:00:00             26.19            26.19            1010   
8   2024-11-19 18:00:00             25.83            26.82            1008   
9   2024-11-19 21:00:00             25.60            26.57            1007   
10  2024-11-20 00:00:00             25.83            26.79            1009   
11  2024-11-20 03:00:00    

# **Parse the data into CSV**
With the Weather data table, we can export it to CSV file and include the current data and time in the filename to keep our files unique and organized.

In [10]:
# Import necessary libraries
import pandas as pd
import os

# Specify the output directory
output_dir = '/Users/yvonne/Downloads'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get the current date and time
current_time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")

# Specify the CSV file to write the data to
csv_file = os.path.join(output_dir, f'weather_data_{current_time}.csv')

# Write weather data to CSV
weather_df.to_csv(csv_file, index=False)

print(f"Weather data has been successfully exported to {csv_file}")

Weather data has been successfully exported to /Users/yvonne/Downloads/weather_data_2024-11-19_00-34-33.csv
